In [2]:
pip install tensorflow_text

     |████████████████████████████████| 4.3MB 8.2MB/s 


In [3]:
import numpy as np
import pandas as pd
import os
import re
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

In [6]:
df = pd.read_csv("final_faq.csv")
df.tail(15)

,Context,Answer
464,How did the first human SARS-CoV-2 infections ...,The first human cases of COVID-19 were identif...
465,Can COVID-19 be spread through coins and bankn...,There is currently no evidence to confirm or d...
466,How can I grocery shop safely in the time of C...,"When grocery shopping, keep at least 1-metre d..."
467,How to wash fruits and vegetables in the time ...,Fruits and vegetables are essential components...
468,Is COVID-19 airborne?,The virus that causes COVID-19 is mainly trans...
469,How long is the incubation period for COVID-19?,The “incubation period” means the time between...
470,What is the state doing to protect our health? \n,California has been actively and extensively p...
471,What is SARS-CoV-2? What is COVID-19?,Severe Acute Respiratory Syndrome Coronavirus-...
472,Where do coronaviruses come from?,Coronaviruses are viruses that circulate among...
473,Is this virus comparable to SARS or to the sea...,The novel coronavirus detected in China is gen...


In [7]:
df = df.loc[1: 400]
df

,Context,Answer
1,What is a coronavirus?,"In humans, several coronaviruses are known to ..."
2,What is COVID-19?,COVID-19 is the infectious disease caused by t...
3,What are the symptoms of COVID-19?,The most common symptoms of COVID-19 are fever...
4,What are the symptoms of COVID-19?,Some people become infected but donâ€™t develo...
5,What are the symptoms of COVID-19?,Around 1 out of every 6 people who gets COVID-...
...,...,...
396,How many people have been infected in the EU/EEA?,See the ECDC daily situation update for the la...
397,How long will this outbreak last?,"Unfortunately, it is not possible to predict h..."
398,Should schools and day centres be closed?,The evidence we have to date indicates that CO...
399,What precautions should I take if I am visitin...,Travellers visiting areas of local or communit...


In [8]:
Answer = list(df.Answer);Answer.append("Hello");Answer.append("Hi, I am Covid-Bot");Answer.append("I am Fine, How can I help u");Answer.append("Good Bye");
Question = list(df.Context);Question.append("Hi");Question.append("Hello");Question.append("How are you ?");Question.append("Bye");

In [9]:
def preprocess_sentences(x):
    return [re.sub(r'(covid-19|covid)', 'coronavirus', i, flags=re.I) for i in x]

In [10]:
# Load module
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

# Create response embeddings
Output = module.signatures['response_encoder'](input=tf.constant(preprocess_sentences(Answer)),context=tf.constant(preprocess_sentences(Question)))['outputs']

In [11]:
pip install flask_ngrok

In [ ]:
from flask import Flask
from flask import Flask, request, render_template
from flask_ngrok import run_with_ngrok
app = Flask(__name__)


run_with_ngrok(app)
@app.route("/")
def home():
    return render_template("home.html")

@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    input_que = []
    input_que.append(userText)
    que = module.signatures['question_encoder'](tf.constant(preprocess_sentences(input_que)))['outputs']
    # return str(dialogue_manager.generate_answer(userText))
    return str(Answer[int(np.argmax(np.inner(que,Output), axis=1))])
 

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://244b02c4bc3f.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Jul/2021 10:01:22] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jul/2021 10:01:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jul/2021 10:01:28] "GET /get?msg=Hi HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Jul/2021 10:01:31] "GET /get?msg=What%20is%20corona%3F HTTP/1.1" 200 -
